<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Chapter 2: Working with Text Data

Packages that are being used in this notebook:

In [43]:
from importlib.metadata import version
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.8.0
tiktoken version: 0.10.0


In [48]:
import os
import urllib.request
import re

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

- (If you encounter an `ssl.SSLCertVerificationError` when executing the previous code cell, it might be due to using an outdated Python version; you can find [more information here on GitHub](https://github.com/rasbt/LLMs-from-scratch/pull/403))

In [49]:
#文本导入
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))

Total number of character: 20479


In [52]:
#标点和字词分词
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

In [ ]:
#建立词汇映射表
vocab = {token:integer for integer,token in enumerate(all_words)}

In [58]:
#分词器
class SimpleTokenizerV1:    
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}#导入词汇映射表
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)                        
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids#编码器，将文本转为token id
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text#解码器：将token id转换为文本

In [59]:
#文本转化为ID
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [60]:
#ID转化为文本
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [62]:
len(vocab.items())

1130

In [63]:
#不存在的新词会报错
tokenizer = SimpleTokenizerV1(vocab)
text = "Hello, do you like tea. Is this-- a test?"
tokenizer.encode(text)

KeyError: 'Hello'

In [ ]:
#添加"<|endoftext|>"token和"<|unk|>"token
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
len(vocab.items())
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

In [ ]:
#编译"<|unk|>"相关的分词器
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [ ]:
#运行示例
tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text3 = "CNN, RNN, CLIP, KAN, U-Net."

text = " <|endoftext|> ".join((text1, text2, text3))

In [ ]:
tokenizer.encode(text)

In [ ]:
tokenizer.decode(tokenizer.encode(text))

In [ ]:
# pip install tiktoken

In [ ]:
import importlib
import tiktoken
print("tiktoken version:", importlib.metadata.version("tiktoken"))

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
#BPE分词器示例
text = (
    "Akwirw ier" 
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

In [ ]:
for idx in integers:
    print(f"ID: {idx}")
    decoded = tokenizer.decode([idx])
    print(f"解码结果: {repr(decoded)}")
    print("---")  # 分隔符

In [ ]:
strings = tokenizer.decode(integers)

print(strings)

In [ ]:
#BPE学习过程详细演示
from tiktoken._educational import *

# 训练一个简单的BPE分词器
def train_simple_encoding():
    # 定义GPT-2的分词模式
    gpt2_pattern = (
        r"""'s|'t|'re|'ve|'m|'ll|'d| ?[\p{L}]+| ?[\p{N}]+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
    )
    # 打开文件时指定encoding="utf-8"
    with open("the-verdict.txt","r", encoding="utf-8") as f:
        data = f.read()
    
    # 训练BPE编码
    enc = SimpleBytePairEncoding.train(data, vocab_size=500, pat_str=gpt2_pattern)
    return enc

# 调用函数
enc = train_simple_encoding()

# 可视化GPT-4编码器如何编码文本
enc = SimpleBytePairEncoding.from_tiktoken("cl100k_base")
enc.encode("hello world aaaaaaaaaaaa")

In [64]:
#重新导入输入数据
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

4690


In [65]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.8.0+cpu


In [66]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # 对全文分词
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"
        
        
        # 使用滑动窗口将文本分块为最大长度（max_length）输入——目标对序列。
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [67]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # 初始化分词器
    tokenizer = tiktoken.get_encoding("gpt2")

    # 导入数据集
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 导入数据加载器
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [69]:
#批量生成批次为8，长度为4的，步长为4的输入——目标对
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]


In [70]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[  287,   262,  6001,   286],
        [  465, 13476,    11,   339],
        [  550,  5710,   465, 12036],
        [   11,  6405,   257,  5527],
        [27075,    11,   290,  4920],
        [ 2241,   287,   257,  4489],
        [   64,   319,   262, 34686],
        [41976,    13,   357, 10915]]), tensor([[  262,  6001,   286,   465],
        [13476,    11,   339,   550],
        [ 5710,   465, 12036,    11],
        [ 6405,   257,  5527, 27075],
        [   11,   290,  4920,  2241],
        [  287,   257,  4489,    64],
        [  319,   262, 34686, 41976],
        [   13,   357, 10915,   314]])]


In [71]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [72]:
#创建一个输入张量
input_ids = torch.tensor([2, 3, 5, 1])

In [73]:
#随机生成一个词汇数量为6，维度为3的嵌入权重矩阵
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [74]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [ ]:
#嵌入向量展示
print(embedding_layer(torch.tensor([3])))
print(embedding_layer(input_ids))

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **You may be interested in the bonus content comparing embedding layers with regular linear layers: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

In [75]:
#生成一个token数位50257，维度为256的嵌入权重矩阵
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- If we sample data from the dataloader, we embed the tokens in each batch into a 256-dimensional vector
- If we have a batch size of 8 with 4 tokens each, this results in a 8 x 4 x 256 tensor:

In [76]:
#构建文本长度为4，批次为4的输入——目标对
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=4, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [77]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257]])

Inputs shape:
 torch.Size([4, 4])


In [78]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)
print(token_embeddings)
# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([4, 4, 4])
tensor([[[-0.2721, -0.3510,  1.1152, -0.6172],
         [-0.7938, -0.0693, -0.8109, -1.3045],
         [-0.3871, -0.4101,  0.8434, -1.0239],
         [-1.1252, -1.0953,  0.5632,  0.6108]],

        [[ 0.1554,  0.3295, -1.0373, -0.2286],
         [ 0.7710,  1.2525,  0.6209, -0.0262],
         [ 0.7711,  0.8476,  0.4660,  0.0256],
         [-0.5001, -0.7551, -2.0384, -0.9043]],

        [[-0.7627, -0.5600, -0.6666,  0.6842],
         [-1.2809,  0.5438,  0.2957, -0.4616],
         [ 0.8227, -1.1443, -0.1711, -0.0343],
         [ 0.1184,  0.6137,  1.2441, -0.9630]],

        [[-0.0309, -0.2959,  0.8625,  0.1213],
         [ 0.0854,  0.9837,  0.4782,  1.3266],
         [-0.2812, -0.6017, -0.0625, -2.4796],
         [ 0.8227, -1.1443, -0.1711, -0.0343]]], grad_fn=<EmbeddingBackward0>)


In [81]:
#生成绝对位置嵌入矩阵
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(max_length))

print(pos_embeddings)
# uncomment & execute the following line to see how the embedding layer weights look like
print(pos_embedding_layer.weight)

tensor([[ 0.5395,  1.8241,  0.5515, -0.4061],
        [-0.8302, -0.7651, -0.8182, -0.1141],
        [-1.3535, -0.5220, -0.1185,  1.4475],
        [ 1.3645,  1.3576,  0.0936,  1.1511]], grad_fn=<EmbeddingBackward0>)
Parameter containing:
tensor([[ 0.5395,  1.8241,  0.5515, -0.4061],
        [-0.8302, -0.7651, -0.8182, -0.1141],
        [-1.3535, -0.5220, -0.1185,  1.4475],
        [ 1.3645,  1.3576,  0.0936,  1.1511]], requires_grad=True)


In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)